In [1]:
import os

import ir_datasets
import evaluate
from tqdm.notebook import tqdm

from irise import INDEX_DIR
from irise.indexer import Indexer
from irise.pipeline import SearchPipeline

2024-06-07 00:39:25.152617: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-07 00:39:26.142093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
SYSTEM = "BM25"
dataset = ir_datasets.load("beir/msmarco/test")
pipe = SearchPipeline()
tf_idf = Indexer()

/home/devrim/anaconda3/envs/irise/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
queries = [q for q in dataset.queries_iter()]

In [4]:
queries[24]

GenericQuery(query_id='855410', text='what is theraderm used for')

In [5]:
predictions = {
    "query": [],
    "q0": [],
    "docid": [],
    "score": [],
    "rank": [],
    "system": [],
}

for query in tqdm(dataset.queries_iter(), total=dataset.queries_count()):
    if query.query_id == "855410":
        continue
    results = tf_idf.search(query.text, weighting="bm25", limit=10, return_ids_only=True)
    if not results:
        predictions["query"].append(int(query.query_id))
        predictions["q0"].append("q0")
        predictions["docid"].append(str(-1))
        predictions["score"].append(-1)
        predictions["rank"].append(-1)
        predictions["system"].append(SYSTEM)
    else:
        for rank, doc_id in enumerate(results):
            predictions["query"].append(int(query.query_id))
            predictions["q0"].append("q")
            predictions["docid"].append(doc_id)
            predictions["score"].append(-1)
            predictions["rank"].append(rank)
            predictions["system"].append(SYSTEM)

  0%|          | 0/43 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
predictions = {
    "query": [],
    "q0": [],
    "docid": [],
    "score": [],
    "rank": [],
    "system": [],
}

for query in tqdm(dataset.queries_iter(), total=dataset.queries_count()):
    if query.query_id == "855410":
        continue
    results = pipe(query.text, weighting="tfidf", return_dict=False)
    if not results:
        predictions["query"].append(int(query.query_id))
        predictions["q0"].append("q0")
        predictions["docid"].append(str(-1))
        predictions["score"].append(-1)
        predictions["rank"].append(-1)
        predictions["system"].append(SYSTEM)
    else:
        for rank, (doc_id, score) in enumerate(results):
            predictions["query"].append(int(query.query_id))
            predictions["q0"].append("q")
            predictions["docid"].append(doc_id)
            predictions["score"].append(score)
            predictions["rank"].append(rank)
            predictions["system"].append(SYSTEM)

  0%|          | 0/43 [00:00<?, ?it/s]

In [6]:
qrels = {
    "query": [],
    "q0": [],
    "docid": [],
    "rel": [],
}
for qrel in tqdm(dataset.qrels_iter(), total=dataset.qrels_count()):
    if qrel.query_id == "855410":
        continue
    qrels["query"].append(int(qrel.query_id))
    qrels["q0"].append("q0")
    qrels["docid"].append(qrel.doc_id)
    qrels["rel"].append(qrel.relevance)

  0%|          | 0/9260 [00:00<?, ?it/s]

In [7]:
trec_eval = evaluate.load("trec_eval")

In [8]:
results_tfidf = trec_eval.compute(references=[qrels], predictions=[predictions])

/home/devrim/anaconda3/envs/irise/lib/python3.11/site-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


In [10]:
results_tfidf_gte = trec_eval.compute(references=[qrels], predictions=[predictions])

/home/devrim/anaconda3/envs/irise/lib/python3.11/site-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


In [8]:
results_bm25_gte = trec_eval.compute(references=[qrels], predictions=[predictions])

/home/devrim/anaconda3/envs/irise/lib/python3.11/site-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


In [9]:
results_bm25_gte

{'runid': 'BM25',
 'num_ret': 420,
 'num_rel': 4098,
 'num_rel_ret': 332,
 'num_q': 42,
 'map': 0.12927758455236976,
 'gm_map': 0.07366230563138433,
 'bpref': 0.13658146912402713,
 'Rprec': 0.13851963010749394,
 'recip_rank': 0.9166666666666666,
 'P@5': 0.8666666666666666,
 'P@10': 0.7904761904761903,
 'P@15': 0.526984126984127,
 'P@20': 0.39523809523809517,
 'P@30': 0.2634920634920635,
 'P@100': 0.07904761904761906,
 'P@200': 0.03952380952380953,
 'P@500': 0.01580952380952381,
 'P@1000': 0.007904761904761906,
 'NDCG@5': 0.7042404043015897,
 'NDCG@10': 0.6718015229180482,
 'NDCG@15': 0.5451254695249996,
 'NDCG@20': 0.4733257303034735,
 'NDCG@30': 0.3992901273944944,
 'NDCG@100': 0.29523932564672967,
 'NDCG@200': 0.2815427515471029,
 'NDCG@500': 0.27952052957091267,
 'NDCG@1000': 0.27952052957091267}

In [11]:
results_tfidf_gte

{'runid': 'BM25',
 'num_ret': 420,
 'num_rel': 4098,
 'num_rel_ret': 238,
 'num_q': 42,
 'map': 0.07917223841863337,
 'gm_map': 0.026082742937849904,
 'bpref': 0.08488730158489216,
 'Rprec': 0.08616219458231825,
 'recip_rank': 0.861904761904762,
 'P@5': 0.6619047619047619,
 'P@10': 0.5666666666666667,
 'P@15': 0.37777777777777777,
 'P@20': 0.2833333333333333,
 'P@30': 0.18888888888888888,
 'P@100': 0.05666666666666668,
 'P@200': 0.02833333333333334,
 'P@500': 0.011333333333333336,
 'P@1000': 0.005666666666666668,
 'NDCG@5': 0.5483813021532049,
 'NDCG@10': 0.49195726864931555,
 'NDCG@15': 0.3966334259409596,
 'NDCG@20': 0.34236886443614756,
 'NDCG@30': 0.285821935610468,
 'NDCG@100': 0.20247154229559539,
 'NDCG@200': 0.19066405294934805,
 'NDCG@500': 0.18890177889577592,
 'NDCG@1000': 0.18890177889577592}

In [7]:
results_tfidf_bm25

{'runid': 'BM25',
 'num_ret': 420,
 'num_rel': 4098,
 'num_rel_ret': 231,
 'num_q': 42,
 'map': 0.07894186507812902,
 'gm_map': 0.025554557457397896,
 'bpref': 0.09278923285848004,
 'Rprec': 0.09648489467718983,
 'recip_rank': 0.7770124716553288,
 'P@5': 0.5904761904761905,
 'P@10': 0.55,
 'P@15': 0.36666666666666664,
 'P@20': 0.275,
 'P@30': 0.18333333333333332,
 'P@100': 0.055,
 'P@200': 0.0275,
 'P@500': 0.011000000000000001,
 'P@1000': 0.0055000000000000005,
 'NDCG@5': 0.4473243356602946,
 'NDCG@10': 0.435539270410992,
 'NDCG@15': 0.35408027174920514,
 'NDCG@20': 0.3079422567448154,
 'NDCG@30': 0.25976833330647775,
 'NDCG@100': 0.19199687783281832,
 'NDCG@200': 0.18338918614092453,
 'NDCG@500': 0.18203178161125239,
 'NDCG@1000': 0.18203178161125239}

In [20]:
results_tfidf_base

{'runid': 'TFIDF',
 'num_ret': 420,
 'num_rel': 4098,
 'num_rel_ret': 61,
 'num_q': 42,
 'map': 0.00733519650322229,
 'gm_map': 0.0002780462054636031,
 'bpref': 0.013481986944545235,
 'Rprec': 0.013568479882912507,
 'recip_rank': 0.25443121693121695,
 'P@5': 0.14761904761904762,
 'P@10': 0.14523809523809522,
 'P@15': 0.09682539682539681,
 'P@20': 0.07261904761904761,
 'P@30': 0.04841269841269841,
 'P@100': 0.014523809523809526,
 'P@200': 0.007261904761904763,
 'P@500': 0.002904761904761905,
 'P@1000': 0.0014523809523809526,
 'NDCG@5': 0.11631847155632262,
 'NDCG@10': 0.11438344107485107,
 'NDCG@15': 0.09050196940136812,
 'NDCG@20': 0.07664279821562887,
 'NDCG@30': 0.06109397810226807,
 'NDCG@100': 0.03576868234321103,
 'NDCG@200': 0.03156786790959604,
 'NDCG@500': 0.030831834471610604,
 'NDCG@1000': 0.030831834471610604}